In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Dataset parameters
dataset_path = 'C:/Users/nisht/Project/ISL_Dataset'  # Replace with your dataset path
IMG_SIZE = 64
BATCH_SIZE = 32

# Data loading and preprocessing
def load_data(dataset_path, img_size=IMG_SIZE):
    images = []
    labels = []
    
    for label_dir in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label_dir)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (img_size, img_size))  # Resize image
                images.append(img)
                labels.append(label_dir)
    
    images = np.array(images) / 255.0  # Normalize images
    labels = np.array(pd.factorize(labels)[0])  # Convert labels to integers
    return images, labels

images, labels = load_data(dataset_path)

# Split data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)


C:\Users\nisht\AppData\Local\Temp\ipykernel_16780\2424267059.py:30: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels = np.array(pd.factorize(labels)[0])  # Convert labels to integers


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D
import numpy as np

# Preparing the data for ConvLSTM
# ConvLSTM expects a 5D tensor: (samples, timesteps, rows, cols, channels)
# Here, we simulate "timesteps" by adding a dummy dimension

def prepare_data_for_convlstm(images, timesteps=1):
    # Adding a time dimension for ConvLSTM (assuming static images treated as one timestep each)
    images = np.expand_dims(images, axis=1)  # Shape: (samples, timesteps, rows, cols, channels)
    return images

X_train_lstm = prepare_data_for_convlstm(X_train)
X_test_lstm = prepare_data_for_convlstm(X_test)

# Model Definition
def build_convlstm(input_shape):
    model = Sequential([
        # ConvLSTM layer
        ConvLSTM2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape, return_sequences=False),
        BatchNormalization(),
        
        # Fully connected layers
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(labels)), activation='softmax')  # Output layer with softmax
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the model
input_shape = (X_train_lstm.shape[1], IMG_SIZE, IMG_SIZE, 3)  # (timesteps, rows, cols, channels)
convlstm_model = build_convlstm(input_shape)

# Train the model
history = convlstm_model.fit(
    X_train_lstm, y_train,
    epochs=20,
    batch_size=BATCH_SIZE,
    validation_data=(X_test_lstm, y_test)
)

# Evaluate the model
y_pred_lstm = np.argmax(convlstm_model.predict(X_test_lstm), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred_lstm))
print("Accuracy Score:", accuracy_score(y_test, y_pred_lstm))

# Save the model
convlstm_model.save('convlstm_model.h5')

C:\Users\nisht\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 754ms/step - accuracy: 0.0622 - loss: 8.5004 - val_accuracy: 0.0284 - val_loss: 3.1474
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 725ms/step - accuracy: 0.0419 - loss: 3.1538 - val_accuracy: 0.0426 - val_loss: 3.1507
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 720ms/step - accuracy: 0.0681 - loss: 3.1199 - val_accuracy: 0.0426 - val_loss: 3.1385
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 721ms/step - accuracy: 0.0710 - loss: 3.0642 - val_accuracy: 0.0284 - val_loss: 3.1307
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 728ms/step - accuracy: 0.1184 - loss: 2.9399 - val_accuracy: 0.0284 - val_loss: 3.1420
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 729ms/step - accuracy: 0.1748 - loss: 2.7361 - val_accuracy: 0.0426 - val_loss: 3.1017
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 738ms/step - accuracy: 0.3406 - loss: 2.2414 - val_accuracy: 0.0638 - val_loss: 3.0885
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 734ms/step - accuracy: 0.5543 - loss: 1.6873 - val_accu

4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/stepWARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000231FFB49440> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step


C:\Users\nisht\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\nisht\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\nisht\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.26      0.60      0.36        10
           2       1.00      0.27      0.43        11
           3       0.57      0.67      0.62         6
           4       1.00      0.45      0.62        11
           5       0.00      0.00      0.00         3
           6       0.71      0.71      0.71         7
           7       0.67      0.67      0.67         6
           8       1.00      0.17      0.29         6
           9       0.42      1.00      0.59         8
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00         1
          12       0.43      0.67      0.52         9
          13       0.67      0.86      0.75         7
          14       0.16      0.75      0.26         4
          15       1.00      0.25      0.40         4
          16       0.67      0.29      0.40         7
   

In [9]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Use the trained ConvLSTM model
model = convlstm_model  # Ensure this is your trained ConvLSTM model

# Define parameters
IMG_SIZE = 64  # Image size used for training
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Z']  # Replace with your actual class labels

# Preprocessing function for live frames
def preprocess_frame(frame):
    # Resize the frame to match the input size of the model
    resized_frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    # Normalize the frame
    normalized_frame = resized_frame / 255.0
    # Add the channel dimension (if grayscale, convert to 3 channels)
    if len(normalized_frame.shape) == 2:
        normalized_frame = np.repeat(normalized_frame[..., np.newaxis], 3, axis=-1)
    # Expand dimensions to match ConvLSTM input: (1, 1, IMG_SIZE, IMG_SIZE, 3)
    return np.expand_dims(np.expand_dims(normalized_frame, axis=0), axis=1)

# Initialize webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Unable to access the webcam.")
    exit()

print("Press 'q' to quit the application.")

# Real-time video processing
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to read frame.")
        break

    # Mirror the frame for a more intuitive interface
    frame = cv2.flip(frame, 1)

    # Draw a rectangle for the region of interest (ROI)
    roi_start = (200, 100)  # Top-left corner of the ROI
    roi_end = (400, 300)    # Bottom-right corner of the ROI
    cv2.rectangle(frame, roi_start, roi_end, (0, 255, 0), 2)

    # Extract the ROI
    roi = frame[roi_start[1]:roi_end[1], roi_start[0]:roi_end[0]]

    # Preprocess the ROI and make predictions
    preprocessed_roi = preprocess_frame(roi)
    try:
        prediction = model.predict(preprocessed_roi)
        predicted_label = class_labels[np.argmax(prediction)]
    except ValueError as e:
        print("Error during prediction:", e)
        predicted_label = "N/A"

    # Display the predicted label
    cv2.putText(frame, f"Prediction: {predicted_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the frame
    cv2.imshow("Sign Language Recognition", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Press 'q' to quit the application.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━